# <center>MegaPrime Image Quality Viewer<center>

In [2]:
import os
import io
from astropy import units as u
from astropy.coordinates import SkyCoord
from IPython.display import Image, display, clear_output
from IPython.core.display import HTML
from astroquery.cadc import Cadc
from astropy.table import Table as ATable
from astropy.io.votable import from_table, writeto
import pandas as pd
import glue_jupyter as gj
import ipywidgets as widgets
from cadcutils import net
from cadctap import CadcTapClient
import base64
import re
import drawSvg as draw


def display_spinner(spinner):
    global d
    if spinner is True:
        display(d)
    else:
        print("")


# Get the list of items for a combobox widgets
#
#
def get_combobox_list(column_name, table_name):
    filter_query = "SELECT DISTINCT {} FROM {}"
    cadc = Cadc()
    output = cadc.exec_sync(filter_query.format(column_name,
                                                table_name))
    if (table_name == 'caom2.distinct_proposal_pi'):
        return [x.lower() for x in list(output[column_name])]
    return list(output[column_name])


def background_color(row):
    color = ''
    for item in range(0, len(out_data.subsets)):
        # Obs. ID is the 4th column
        if row.values[3] in out_data.subsets[item]['Obs. ID']:
            color = color_list[item % 11]
    return ['background-color: %s ' % color] * len(row.values)


def subset_background_color(row, color):
    return ['background-color: %s ' % color] * len(row.values)


# -------------------page clicked-------------------------- #
def subset_page_clicked(b):
    global subset_page
    with out3:
        clear_output()
        num_pages = int(len(subset_df.index) / 10)
        if b.description == 'First':
            subset_page = 0
        elif b.description == 'Previous':
            if subset_page != 0:
                subset_page -= 1
        elif b.description == 'Next':
            if subset_page != num_pages:
                subset_page += 1
        elif b.description == 'Last':
            subset_page = num_pages
        page_df = subset_df[subset_page * 10: subset_page * 10 + 10]
        display(page_df.style.apply(subset_background_color,
                                    color=color_list[selected_color % 11],
                                    axis=1))


# -------------------page clicked--------------------------- #
def table_clicked(b):
    global table_page
    with out2:
        clear_output()
        num_pages = int(len(results_df.index) / 10)
        if b.description == 'First':
            table_page = 0
        elif b.description == 'Previous':
            if table_page != 0:
                table_page -= 1
        elif b.description == 'Next':
            if table_page != num_pages:
                table_page += 1
        elif b.description == 'Last':
            table_page = num_pages
        sub_df = results_df[table_page * 10: table_page * 10 + 10]
        display(sub_df.style.apply(background_color, axis=1))


def select_method(obtain_images):
    if (obtain_images == 'Download Images'):
        txt = ""
        for key in url_dictionary:
            txt = txt + str(key) + '\n'
        b64 = base64.b64encode(txt.encode())
        payload = b64.decode()
        html = """<a download="testfile" href="data:text;base64,{payload}"
        target="_blank">Download Subset</a>"""
        html = html.format(payload=payload)
        display(HTML(html))
    else:
        if len(url_dictionary) > 15:
            print("Can not display more than 15 images")
        else:
            for value in url_dictionary.values():
                print("Observation Id: {}".format(value[1]))
                link = '<a href="{}">{}</a>'.format(str(value[0]),
                                                    str(value[0]))
                display(HTML(link))
                display(Image(url=str(value[0]),
                              width=100,
                              height=100,
                              unconfined=True))


def preview_subset(subset):
    global url_dictionary, Obtain_Img, subset_df, selected_color
    lable = subset.split(' ')
    i = int(lable[1]) - 1
    subset_data = {
        'Preview': out_data.subsets[i]['Preview'],
        'publisherID': out_data.subsets[i]['Publisher ID'],
        'Collection': out_data.subsets[i]['Collection'],
        'Obs. ID': out_data.subsets[i]['Obs. ID'],
        'Product ID': out_data.subsets[i]['Product ID'],
        'Instrument': out_data.subsets[i]['Instrument'],
        'Int. Time': out_data.subsets[i]['Int. Time'],
        'Overall Quality': out_data.subsets[i]['Overall Quality'],
        'Really Bad Tracking': out_data.subsets[i]['Really Bad Tracking'],
        'Bad Tracking': out_data.subsets[i]['Bad Tracking'],
        'Bad Weather': out_data.subsets[i]['Bad Weather'],
        'Background Problem': out_data.subsets[i]['Background Problem'],
        'Dead CCDs': out_data.subsets[i]['Dead CCDs']}
    subset_df = pd.DataFrame(subset_data)
    cadc = Cadc()
    subset_table = ATable.from_pandas(subset_df)
    url_dictionary = {}
    for idx in range(len(subset_df)):
        url = cadc.get_data_urls(subset_table[idx: idx + 1],
                                 include_auxiliaries=True)
        fz_url = next((u for u in url if '.fz' in u), None)
        jpg_url = next((u for u in url if '1024.jpg' in u), None)
        if jpg_url:
            url_dictionary[fz_url] = [jpg_url, subset_df['Obs. ID'][idx]]
    if(subset_num_records[i] > 0):
        print("Number of record: {}".format(subset_num_records[i]))
        selected_color = i
        # --------- here to add the buttons ----------- #
        previous_button = widgets.Button(
            description="Previous",
            icon='backward',
            style=widgets.ButtonStyle(button_color='#E58975'),
            layout=widgets.Layout(flex='1 1 auto',
                                  width='auto'))
        next_button = widgets.Button(
            description="Next",
            icon='forward',
            style=widgets.ButtonStyle(button_color='#E58975'),
            layout=widgets.Layout(flex='1 1 auto',
                                  width='auto'))
        first_button = widgets.Button(
            description="First",
            icon='step-backward',
            style=widgets.ButtonStyle(button_color='#E58975'),
            layout=widgets.Layout(flex='1 1 auto',
                                  width='auto'))
        last_button = widgets.Button(
            description="Last",
            icon='step-forward',
            style=widgets.ButtonStyle(button_color='#E58975'),
            layout=widgets.Layout(flex='1 1 auto',
                                  width='auto'))
        previous_button.on_click(subset_page_clicked)
        next_button.on_click(subset_page_clicked)
        first_button.on_click(subset_page_clicked)
        last_button.on_click(subset_page_clicked)
        button_ui = widgets.HBox([first_button,
                                  previous_button,
                                  next_button,
                                  last_button])
        first_button.click()
        display(button_ui, out3)
        obtain_images = widgets.Dropdown(
            options=['Download Images',
                     'Display Images'],
            description='Obtain Images',
            style={'description_width': 'initial'})
        selection = widgets.HBox([obtain_images])
        select_output = widgets.interactive_output(
            select_method,
            {'obtain_images': obtain_images})
        display(selection, select_output)


def click_load_subsets(b):
    global subset_num_records, click_count
    with out:
        click_count = click_count + 1
        if(click_count % 2 == 0):
            clear_output()
        else:
            clear_output()
            if(len(out_data.subsets) != 0):
                record_list = []
                subset_num_records = []
                for i in range(len(out_data.subsets)):
                    record_list.append(out_data.subsets[i].label)
                    n_records = len(out_data.subsets[i]['Product ID'])
                    subset_num_records.append(n_records)

                subset_dropdown = widgets.Dropdown(
                    options=record_list,
                    description='Select Subset',
                    layout=widgets.Layout(flex='1 1 auto',
                                          width='auto'),
                    style={'description_width': 'initial'})

                subset_ui_style = widgets.Layout(
                    display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='100%')

                subset_ui = widgets.Box(children=[subset_dropdown],
                                        layout=subset_ui_style)

                subset_output = widgets.interactive_output(
                    preview_subset,
                    {'subset': subset_dropdown})

                display(subset_ui, subset_output)
            else:
                print("\x1b[31m no subset created \x1b[0m")


def graphs(G_Type):
    global out_data, results_df
    if (G_Type == 'Scatter'):
        scatter_viewer = app.scatter2d(x='Really Bad Tracking',
                                       y='Overall Quality',
                                       data=out_data,
                                       show=True)
    elif (G_Type == 'Histogram'):
        histogram_viewer = app.histogram1d(x='Overall Quality',
                                           data=out_data,
                                           show=True)
    else:
        previous_button = widgets.Button(
            description="Previous",
            icon='backward',
            style=widgets.ButtonStyle(button_color='#E58975'),
            layout=widgets.Layout(flex='1 1 auto',
                                  width='auto'))

        next_button = widgets.Button(
            description="Next",
            icon='forward',
            style=widgets.ButtonStyle(button_color='#E58975'),
            layout=widgets.Layout(flex='1 1 auto',
                                  width='auto'))

        first_button = widgets.Button(
            description="First",
            icon='step-backward',
            style=widgets.ButtonStyle(button_color='#E58975'),
            layout=widgets.Layout(flex='1 1 auto',
                                  width='auto'))

        last_button = widgets.Button(
            description="Last",
            icon='step-forward',
            style=widgets.ButtonStyle(button_color='#E58975'),
            layout=widgets.Layout(flex='1 1 auto',
                                  width='auto'))

        previous_button.on_click(table_clicked)
        next_button.on_click(table_clicked)
        first_button.on_click(table_clicked)
        last_button.on_click(table_clicked)
        button_ui = widgets.HBox([first_button,
                                  previous_button,
                                  next_button,
                                  last_button])
        first_button.click()
        display(button_ui, out2)


def start(search_button):
    global app, out_data, results_df
    query_widget_1.value = ""
    if(space_object.value == ""):
        print("\x1b[31m No target object selected \x1b[0m")
        return 0
    spinner = widgets.Checkbox(value=True, description='')
    output_spinner = widgets.interactive_output(display_spinner,
                                                {'spinner': spinner})
    display(output_spinner)
    radius = 0.01666666666666
    splited = space_object.value.split(' ')
    if(re.search(r'^[a-zA-Z0-9_]+$', splited[0])):
        if len(splited) >= 1:
            object_name = splited[0]
            try:
                coords = SkyCoord.from_name(object_name)
                ra_degree = coords.ra.degree
                dec_degree = coords.dec.degree
            except Exception:
                print("\x1b[31m Invalid target object \x1b[0m")
                spinner.value = False
                return 0
            if(len(splited) > 1 and re.search(r'^\d+\.?\d*$', splited[1])):
                radius = float(splited[1])
            elif(len(splited) > 1 and re.search(r'arcmin', splited[1])):
                radius = float(float(splited[1][:-6]) / 60)
            elif(len(splited) > 1 and re.search(r'^\d+\'$', splited[1])):
                radius = float(float(splited[1][:-1]) / 60)

    elif(re.search(r'^-?\d+\.?\d*$', splited[0])):
        if len(splited) >= 2:
            if(re.search(r'^-?\d+\.?\d*$', splited[1])):
                ra_degree = float(splited[0])
                dec_degree = float(splited[1])
            else:
                print("\x1b[31m Invalid target object \x1b[0m")
                spinner.value = False
                return 0
            if(len(splited) > 2 and re.search(r'^\d+\.?\d*$', splited[2])):
                radius = float(splited[2])
        else:
            spinner.value = False
            return 0
    elif(re.search(r'^\+?-?\d+\:\d+\:\d+\.?\d*$', splited[0])):
        if len(splited) >= 2:
            if(re.search(r'^\+?-?\d+\:\d+\:\d+\.?\d*$', splited[1])):
                coords = SkyCoord(splited[0], splited[1],
                                  unit=(u.hourangle, u.deg))
                ra_degree = coords.ra.degree
                dec_degree = coords.dec.degree
            else:
                print("\x1b[31m Invalid target object \x1b[0m")
                spinner.value = False
                return 0
            if(len(splited) > 2 and re.search(r'^\d+\.?\d*$', splited[2])):
                radius = float(splited[2])
        else:
            spinner.value = False
            return 0
    else:
        spinner.value = False
        return 0
    obs_query = """SELECT Observation.observationURI,
    Plane.publisherID,
    Observation.collection,
    Observation.observationID,
    Plane.productID,
    Observation.instrument_name,
    Plane.time_exposure
    FROM caom2.Plane AS Plane
    JOIN caom2.Observation AS Observation
    ON Plane.obsID = Observation.obsID
    WHERE (collection = '{collection}'
    AND instrument_name = '{instrument}' """
    obs_query_param = {'collection': 'CFHT',
                       'instrument': 'MegaPrime'}

    if(space_object.value != ""):
        obs_query = obs_query + """AND INTERSECTS(CIRCLE('{epoch}',
        {ra}, {dec}, {radius}), Plane.position_bounds) = 1"""
        obs_query_param['epoch'] = 'ICRS'
        obs_query_param['ra'] = ra_degree
        obs_query_param['dec'] = dec_degree
        obs_query_param['radius'] = radius
    if(bandpassName.value != ""):
        obs_query = obs_query + """ AND
        Plane.energy_bandpassName = '{bandpassName}'"""
        obs_query_param['bandpassName'] = bandpassName.value
    if(pi.value != ""):
        obs_query = obs_query + """ AND
        lower(Observation.proposal_pi) LIKE '%{pi}%'"""
        obs_query_param['pi'] = pi.value

    obs_query = obs_query + ")"
    # assign the query to accordion
    query_widget_1.value = obs_query.format(**obs_query_param)
    output_file = 'tmp/test_vooutput.xml'
    cadc = Cadc()
    output = cadc.exec_sync(obs_query.format(**obs_query_param))
    if (len(output) == 0):
        print(" No Record from first query . ")
        spinner.value = False
        return 0
    votable = from_table(output)
    writeto(votable, "tmp/output.xml")
    anonSubject = net.Subject()
    # client=CadcTapClient(anonSubject,resource_id='ivo://cadc.nrc.ca/youcat')
    certSubject = net.Subject(
        certificate=os.path.join(os.environ['HOME'], ".ssl/cadcproxy.pem"))
    client = CadcTapClient(certSubject)
    quality_query = """SELECT Tmp.observationURI AS Preview,
    Tmp.publisherID AS "Publisher ID",
    Tmp.collection AS Collection,
    Tmp.observationID AS "Obs. ID",
    Tmp.productID AS "Product ID",
    Tmp.instrument_name AS Instrument,
    Tmp.time_exposure AS "Int. Time",
    Quality.overallQuality AS "Overall Quality",
    Quality.reallyBadTracking AS "Really Bad Tracking",
    Quality.badTracking AS "Bad Tracking",
    Quality.badWeather AS "Bad Weather",
    Quality.backgroundProblem AS "Background Problem",
    Quality.deadCCDs AS "Dead CCDs"
    FROM ml.MegaprimeQuality AS Quality
    JOIN tap_upload.tmptable AS Tmp
    ON Quality.observationID=Tmp.observationID
    WHERE publisherID LIKE '%p'"""
    def_table = os.path.join('tmp', 'output.xml')
    out_file = io.StringIO()
    client.query(quality_query,
                 response_format='csv',
                 output_file=out_file,
                 tmptable='tmptable:' + def_table, timeout=30)
    out_file.seek(0)
    results_df = pd.read_csv(out_file)
    results_df.columns = [col.replace('"', '') for col in results_df.columns]
    results_df.dropna(subset=['Obs. ID'], inplace=True)
    results_df.set_index('Obs. ID', inplace=True)  # remove the index column
    count_row = len(results_df.index)
    if (count_row == 0):
        print(" No Record from the second query . ")
        spinner.value = False
        return 0
    print('Total row count: {}'.format(str(count_row)))
    # display data using glue
    app = gj.jglue()
    out_data = app.load_data(results_df.to_csv())
    results_df.reset_index(inplace=True)   # get the old index column back
    graph_output = widgets.interactive_output(graphs, {'G_Type': G_Type})
    display(ui_graph, graph_output)
    # display the button to load subsets
    view_subset_button = widgets.Button(
        description="View Subsets",
        style=widgets.ButtonStyle(button_color='#E58975'),
        layout=widgets.Layout(flex='1 1 auto', width='auto'))

    view_subset_button.on_click(click_load_subsets)
    button_layout = widgets.Layout(display='flex',
                                   flex_flow='row',
                                   align_items='stretch',
                                   width='100%')
    button_auto = widgets.Box(children=[view_subset_button],
                              layout=button_layout)
    view_subset_button.click()
    display(widgets.VBox([button_auto, out]))
    spinner.value = False


# ------------------------starting point--------------------- #
click_count = 0
out = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
info = """

Positions entered as RA & DEC in degrees or sexagesimal.
Target name resolution uses any resolvers in the drop-down
menu or the original target designation of the observation
if the "Deselect to query by string" checkbox is deselected.

Optional search radius (default unit is degrees) can be
specified. If not specified a default radius of 1 arcminute
is used. A coordinate system/epoch (ICRS [default], J2000,
J2000.0, B1950, B1950.0, FK4, FK5, GAL) can be appended.

A range of RA and DEC or l and b may be specified using
".." syntax.

If the "Do Spatial Cutout" box is selected then the diameter
of the image cutout is twice the specified radius.
Box labelled "Choose File" for multiple target file upload
with one target per row.

Clickable examples:
    -M101
    -M101 0.5
    -M101 30arcmin
    -M101 30'
    -HD79158 0.05
    -210.05 54.3
    -210.05 54.3 0.5
    -210.05 54.3 0.5 FK4
    -08:45:07.5 +54:18:00
    -08:45:07.5 +54:18:00 0.5
    -08 45 07.5 +54 18 00 0.5

To search for observations of solar system bodies follow the link to SSOIS.
"""

d = draw.Drawing(800, 500, origin='center')
d.append(draw.Circle(45, -5, 5,
                     fill='red',
                     fill_opacity=0.5))
d.append(draw.Circle(40, -22, 5,
                     fill='red',
                     fill_opacity=0.5))
d.append(draw.Circle(20, -30, 5,
                     fill='red',
                     fill_opacity=0.5))
d.append(draw.Circle(4, -22, 5,
                     fill='red',
                     fill_opacity=0.5))
d.append(draw.Circle(-5, -5, 5,
                     fill='red'))
d.append(draw.Circle(4, 13, 5,
                     fill='red'))
d.append(draw.Circle(20, 20, 5,
                     fill='red'))
d.append(draw.Circle(37, 13, 5,
                     fill='red',
                     fill_opacity=0.5))

space_object = widgets.Text(
    value="",
    description="Target Object/Target Coordinates",
    continuous_update=True,
    layout=widgets.Layout(flex='1 1 auto', width='auto'),
    style={'description_width': '31.5%'})

info_button = widgets.ToggleButton(
    description='',
    disabled=True,
    tooltip=info,
    icon='info',
    layout=widgets.Layout(width='4%'))
space_object_ui = widgets.Box([space_object, info_button])

bandpassName = widgets.Combobox(
    value='',
    placeholder='',
    options=get_combobox_list('energy_bandpassName', 'caom2.EnumField'),
    description='Filter',
    layout=widgets.Layout(flex='1 1 auto', width='auto'),
    style={'description_width': '30%'},
    disabled=False)

pi = widgets.Combobox(
    value='',
    placeholder='',
    options=get_combobox_list('proposal_pi', 'caom2.distinct_proposal_pi'),
    description='P.I. Name',
    layout=widgets.Layout(flex='1 1 auto', width='auto'),
    style={'description_width': '30%'},
    disabled=False)

search_button = widgets.ToggleButton(
    value=False,
    description="Search",
    button_style='',
    button_color='#E58975',
    layout=widgets.Layout(flex='1 1 auto', left='300px', width='50%'))

box_layout = widgets.Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='80%')

tab1_ui = widgets.Box([space_object_ui, bandpassName, pi, search_button],
                      layout=box_layout)

query_widget_1 = widgets.HTML(
    value="",
    placeholder='query 1',
    description='')

query_accordion = widgets.Accordion(children=[query_widget_1])
query_accordion.set_title(0, 'ADQL')

color_list = ['#F98DDA', '#C7E6A1', '#A1E6E2', '#FAFBA1',
              '#FF9C33', '#CBC6C0', '#D8A56E', '#F7A480',
              '#D798EE', '#80A0C8', '#FACCED']

G_Type = widgets.Dropdown(
    options=['Scatter', 'Histogram', 'Table'],
    description='View Data',
    style={'description_width': 'initial'})
ui_graph = widgets.HBox([G_Type])

database_output = widgets.interactive_output(
    start,
    {"search_button": search_button})

tab = widgets.Tab(children=[tab1_ui,
                            query_accordion,
                            database_output])
tab.set_title(0, 'Space Object')
tab.set_title(1, 'ADQL')
tab.set_title(2, 'Result')

display(tab)